<a href="https://colab.research.google.com/github/edoardomarsigli/FRO_Labs_22-23/blob/master/MarsigliSvanella.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Run this cell to download all instances from the GitHub repository.
# It also downloads results for instances 1, 2, 3, and 4.

# Clean files if present
!rm instance_1.json
!rm result_1.txt
!rm instance_2.json
!rm result_2.txt
!rm instance_3.json
!rm result_3.txt
!rm instance_4.json
!rm result_4.txt
!rm instance_5.json
!rm instance_6.json
!rm instance_7.json
!rm instance_8.json

# Download directly from Github
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_1.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_2.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_3.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_4.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_5.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_6.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_7.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_8.json

!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/solutions/result_1.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/solutions/result_2.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/solutions/result_3.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/solutions/result_4.txt

--2023-06-06 21:29:09--  https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 968 [text/plain]
Saving to: ‘instance_1.json’

instance_1.json     100%[===================>]     968  --.-KB/s    in 0s      

2023-06-06 21:29:09 (82.5 MB/s) - ‘instance_1.json’ saved [968/968]

--2023-06-06 21:29:09--  https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 

In [23]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import json
import math
import datetime
import numpy as np
import mip
import networkx as nx

# Reads a .json instance and returns it in a dictionary
def load_instance(filename):
  with open(filename, 'r') as f:
    data = json.load(f)
  return data

def load_result(filename):
  try:
    with open(filename, 'r') as f:
       return float(f.read())
  except FileNotFoundError:
    return None

In [25]:
def get_cycles(V, E, x):
    graph = [[] for i in V]

    for (i,j) in E:
      if x[i,j].x > 0.5:
        graph[i].append(j)

    cycles = []
    color = [0 for i in V]
    par = [-1 for i in V]

    for i in V:
      if par[i] == -1:
        dfs_cycle(graph, cycles, i, -1, color, par)

    for u in V:
      adj = [v for v in graph[u] if v > u]
      for v in adj:
        if u in graph[v]:
          cycles.append([u,v])

    return [c for c in cycles if len(c) < len(V) and len(c) > 0 and not 0 in c]

  # depth first search algorithm
def dfs_cycle(graph, cycles, u, p, color, par):
    # skip completely visited vertex.
    if color[u] == 2:
        return
 
    # If vertex has been seen but not completely visited -> cycle detected.
    # Backtrack based on parents to find the complete cycle.
    if color[u] == 1:
        v = []
        cur = p
        v.append(cur)
 
        # backtrack the vertex which are
        # in the current cycle thats found
        while cur != u:
            cur = par[cur]
            v.append(cur)
        
        cycles.append(v)
        return
 
    par[u] = p
 
    # partially visited.
    color[u] = 1
 
    # simple dfs on graph
    for v in graph[u]:
        # if it has not been visited previously
        if v == par[u]:
            continue
        dfs_cycle(graph, cycles, v, u, color, par)
 
    # completely visited.
    color[u] = 2



In [26]:
def solve(instance): #soluzione con powerset
  
  depot_coordinates = np.array(instance["depot_coordinates"])
  town_coordinates = np.array(instance["town_coordinates"])
  teacher_coordinates = np.array(instance["teacher_coordinates"])
  students_per_town = np.array(instance["students_per_town"])
  bus_capacity = instance["bus_capacity"]
  nstudents= [0]

  depot_town_coordinates= np.concatenate((depot_coordinates,town_coordinates))
  tot_coordinates= np.concatenate((depot_coordinates, town_coordinates, teacher_coordinates))
  nstudents_coordinates = np.concatenate((nstudents, students_per_town))

  V = range(len(tot_coordinates))
  V0 = V[1:]
  Vc = V[1:len(town_coordinates)+1]
  K= range(len(teacher_coordinates))
  U= V[len(town_coordinates)+1:]

  
  A = [(i,j) for i in V for j in V if i!=j]

  distance = np.array([[math.sqrt(np.sum((tot_coordinates[i] - tot_coordinates[j])**2)) for i in V] for j in V])
  coord = {i: (tot_coordinates[i,0], tot_coordinates[i,1]) for i in V}
  g = nx.Graph()
  g.add_nodes_from(V)
  g.add_edges_from(A)
  nx.draw(g, coord, with_labels=True, node_color="blue")

  m = mip.Model()
  
  x = {(i,j, k): m.add_var(var_type=mip.BINARY) for (i,j) in A for k in K}
  #x = {(i,j, k): m.add_var(var_type=mip.BINARY, name="x{}{}".format(str(i), str(j), k)) for (i,j) in E for k in K if i!=j}
       
  y =  {(i,k): m.add_var(var_type=mip.BINARY) for i in V for k in K}
  
  u = {(i): m.add_var(var_type=mip.BINARY) for i in U}

  #CONSTRAINS:
  for i in V:
    for z in K:
      m.add_constr(mip.xsum(x[i,j,z]  for j in V if (i,j) in A)==mip.xsum(x[j,i,z] for j in V if (i,j) in A))
      m.add_constr(mip.xsum(x[i,j,z]  for j in V if (i,j) in A)==y[i,z])
  
  for i in Vc:
        m.add_constr(mip.xsum(y[i,k] for k in K) == 1)
  
  m.add_constr(mip.xsum(y[0,k] for k in K) == mip.xsum(u[i] for i in U))
  
  for k in K:
    m.add_constr(mip.xsum(nstudents_coordinates[i] * y[i,k] for i in Vc)<= bus_capacity)
 
  for i in U:
    m.add_constr(mip.xsum(y[i,z] for z in K) == (1- u[i]))
 

  from itertools import chain, combinations
  powerset = list(chain.from_iterable(combinations(V0, r) for r in V0))

  for S in powerset:
    if len(S) >= 1:
        for k in K:
          m.add_constr(mip.xsum(x[i,j,k] for i in S for j in S if (i,j) in A) <= len(S)-1)

  
  #FO:
  m.objective = mip.minimize(mip.xsum(distance[i,j] * mip.xsum(x[i,j,k] for k in K) for (i,j) in A) + mip.xsum(distance[k,0] * u[k] for k in U))
  
  m.optimize()

  g.clear()
  g.add_edges_from([(i,j) for (i,j) in A if x[i,j,0].x > 0.001])
  nx.draw(g, pos=coord, width=4, edge_color='red')

  g.clear()
  g.add_edges_from([(i,j) for (i,j) in A if x[i,j,1].x > 0.001])
  nx.draw(g, pos=coord, width=5, edge_color='green')

  if k > 2:
    g.clear()
    g.add_edges_from([(i,j) for (i,j) in A if x[i,j,2].x > 0.001])
    nx.draw(g, pos=coord, width=5, edge_color='blue')
 
  return m.objective_value




In [27]:
def solve1(instance):  #soluzione con getcycle
  
  depot_coordinates = np.array(instance["depot_coordinates"])
  town_coordinates = np.array(instance["town_coordinates"])
  teacher_coordinates = np.array(instance["teacher_coordinates"])
  students_per_town = np.array(instance["students_per_town"])
  bus_capacity = instance["bus_capacity"]
  nstudents= [0]
  
  K= range(len(teacher_coordinates))
  t = [1 for i in K]
  depot_town_coordinates= np.concatenate((depot_coordinates,town_coordinates))
  tot_coordinates= np.concatenate((depot_coordinates, town_coordinates, teacher_coordinates))
  nstudents_coordinates = np.concatenate((nstudents, students_per_town, t))

  

  V = range(len(tot_coordinates))
  V0 = V[1:]
  Vc = V[1:len(town_coordinates)+1]
  
  U= V[len(town_coordinates)+1:]

  
  A = [(i,j) for i in V for j in V if i!=j]
  distance = np.array([[math.sqrt(np.sum((tot_coordinates[i] - tot_coordinates[j])**2)) for i in V] for j in V])

  def r(c):
    somma = sum([nstudents_coordinates[i] for i in c])
    return int(np.ceil(somma/bus_capacity))

  m = mip.Model()
  
  x = {(i,j, k): m.add_var(var_type=mip.BINARY) for (i,j) in A for k in K}
  #x = {(i,j, k): m.add_var(var_type=mip.BINARY, name="x{}{}".format(str(i), str(j), k)) for (i,j) in E for k in K if i!=j}
       
  y =  {(i,k): m.add_var(var_type=mip.BINARY) for i in V for k in K}
  
  u = {(i): m.add_var(var_type=mip.BINARY) for i in U}

  #CONSTRAINS:

  for i in V:
    for z in K:
      m.add_constr(mip.xsum(x[i,j,z]  for j in V if (i,j) in A)==mip.xsum(x[j,i,z] for j in V if (i,j) in A))
      m.add_constr(mip.xsum(x[i,j,z]  for j in V if (i,j) in A)==y[i,z])
  
  for i in Vc:
        m.add_constr(mip.xsum(y[i,k] for k in K) == 1)
  
  m.add_constr(mip.xsum(y[0,k] for k in K) == mip.xsum(u[i] for i in U))
  
  for k in K:
    m.add_constr(mip.xsum(nstudents_coordinates[i] * y[i,k] for i in Vc)<= bus_capacity)
 
  for i in U:
    m.add_constr(mip.xsum(y[i,z] for z in K) == (1- u[i]))
      
  m.add_constr(mip.xsum(x[0,j,k] for j in V for k in K if j!=0) == r(V))
  m.add_constr(mip.xsum(x[i,0,k] for i in V for k in K if i!=0) == r(V))

  m.objective = mip.minimize(mip.xsum(distance[i,j] * mip.xsum(x[i,j,k] for k in K) for (i,j) in A) + mip.xsum(distance[k,0] * u[k] for k in U))
  
  m.optimize()

  w = {(i,j): m.add_var(var_type=mip.BINARY) for (i,j) in A}
  for (i,j) in A:
      w[i,j] = mip.xsum(x[i,j,k] for k in K)

  cycles = get_cycles(V, A, w)
  while len(cycles)>0:
    for cycle in cycles:
      cycle_edges = [w[i,j] for (i,j) in A if i in cycle and j in cycle]
      m += mip.xsum(cycle_edges) <= len(cycle) - r(cycle)

      m.optimize()
      cycles = get_cycles(V, A, w)
 
  return m.objective_value


In [28]:
insts = [1, 2, 3, 4, 5, 6, 7, 8]
insts1 = [1, 2, 3]
outputs = []
times = []

for i in insts1:
  instance_name = "instance_{}.json".format(i)
  results_name = "result_{}.txt".format(i)
  inst = load_instance(instance_name)

  # start and end are used to measure execution time
  start = datetime.datetime.now()

  #ottimizzo con la funzione powerset le prime 3 istanze
  
  obj = solve1(inst)
  
  end = datetime.datetime.now()
  time = (end - start).total_seconds()

  print("[{}] Found solution with obj: {}".format(i, obj))
  print("[{}] Elapsed time: {} s".format(i, time))


  res = load_result(results_name)
  if res is not None:
    gap = 100 * (obj - res) / res
    print("[{}] Gap: {}".format(i, gap))

  outputs.append(obj)
  times.append(time)

insts2 = [4, 5, 6, 7, 8]
for i in insts2:
  instance_name = "instance_{}.json".format(i)
  results_name = "result_{}.txt".format(i)
  inst = load_instance(instance_name)

  # start and end are used to measure execution time
  start = datetime.datetime.now()

  #ottimizzo con la funzione getcycle le ultime 5 istanze (per stare sotto i 30 min)
  
  obj = solve1(inst)
  
  end = datetime.datetime.now()
  time = (end - start).total_seconds()

  print("[{}] Found solution with obj: {}".format(i, obj))
  print("[{}] Elapsed time: {} s".format(i, time))


  res = load_result(results_name)
  if res is not None:
    gap = 100 * (obj - res) / res
    print("[{}] Gap: {}".format(i, gap))

  outputs.append(obj)
  times.append(time)


print("Id,Obj,Time")
for i in range(len(insts)):
  print(insts[i], outputs[i], times[i])

[1] Found solution with obj: 458.27729249529904
[1] Elapsed time: 2.558083 s
[1] Gap: -1.2403717092613988e-14
[2] Found solution with obj: 712.5038409416293
[2] Elapsed time: 7.089968 s
[2] Gap: 1.5955961384203916e-14
[3] Found solution with obj: 414.8740205020167
[3] Elapsed time: 15.946605 s
[3] Gap: -1.3701368620774288e-14
Id,Obj,Time
1 458.27729249529904 2.558083
2 712.5038409416293 7.089968
3 414.8740205020167 15.946605


IndexError: ignored